# FRAUD DETECTION MATRICES CALCUATION

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import time 
import numpy as np 
import seaborn as sns 
import datetime


## Importing The dataset

In [2]:
#Importing data set
march_orders = pd.read_csv('H:/intern_lovelocal/Fraud Analysis/march_orders.csv')
march_order_products= pd.read_csv('H:/intern_lovelocal/Fraud Analysis/march_order_products.csv')
march_order_product_details = pd.read_csv('H:/intern_lovelocal/Fraud Analysis/march_order_product_details.csv')
march_customer_device_mapping = pd.read_csv('H:/intern_lovelocal/Fraud Analysis/march_customer_device_mapping.csv')

In [11]:
march_orders.head()

,order_id,customer_id,store_id,pickup_from_store,delivery_status,created_on,version,delivery_address,address_type,customer_lat,customer_long,address_text,bill_total,amount_payable,discount,store_lat,store_long
0,10005527,d205ba8c97144daca7727c732bca41f2,539786,1,DELIVERED,2020-03-20 09:23:51.975199,0,{},NaN,NaN,NaN,NaN,200.0,100.0,0.0,19.158317,72.841473
1,10017342,f08492a1f6ae4e02884d7ea45f832633,428028,1,DELIVERED,2020-03-13 09:11:42.537974,0,{},NaN,NaN,NaN,NaN,220.0,120.0,0.0,19.159133,72.849293
2,10023158,696ea6c5e87f46e7866cc4d8fe66123b,428028,1,DELIVERED,2020-03-01 10:32:44.458359,0,{},NaN,NaN,NaN,NaN,219.0,119.0,0.0,19.159133,72.849293
3,10032101,8d65aeaf22e84686ab6b2571df1aa51f,744294,1,DELIVERED,2020-03-17 11:19:21.143282,0,{},NaN,NaN,NaN,NaN,216.0,116.0,0.0,19.163073,72.838832
4,10047224,fc4b43de31b947e78a4f34372b536c54,63308,1,DELIVERED,2020-03-20 05:37:12.203217,0,{},NaN,NaN,NaN,NaN,266.0,166.0,0.0,19.227747,72.849731


In [12]:
march_order_products.head()

,order_id,version,mpaani_product_id,store_product_id,quantity,fulfilable
0,10005527,0,39765,1969747,1,1
1,10005527,0,33016,1968760,7,1
2,10017342,0,521993,1992216,4,1
3,10023158,0,184822,1555197,1,1
4,10023158,0,522083,1992854,5,1


In [13]:
march_order_product_details.head()

,mpaani_product_id,product_name,mrp,measurement_unit,measurement_value
0,32751,Kurkure Solid Masti (19 g),5.0,NaN,0.0
1,32752,Kellogs Chocos Chota Laddoo (26 g),10.0,NaN,0.0
2,32753,Haldiram's Potato Lite (17 g),5.0,NaN,0.0
3,32754,Haldiram's Khatta Meetha (30 g),5.0,NaN,0.0
4,32756,Haldiram's Pancharatan Mixture (25 g),10.0,NaN,0.0


In [14]:
march_customer_device_mapping.head()

,customer_id,device_identifier,created_on,modified_on
0,c0f42047fe384b199c7fb216da5f1c25,67429302aad14e89,2020-03-19 07:58:27.580351,2020-03-19 07:58:27.580408
1,c0f42047fe384b199c7fb216da5f1c25,cbf03201e3a2e4f4,2020-03-20 08:28:35.139651,2020-03-20 08:28:35.139715
2,53c120de0095445ea9e02652f27c62f9,67429302aad14e89,2020-03-20 10:45:09.073413,2020-03-20 10:45:09.073476
3,e2daa7d8d19b49bf8e86e6a53d8bf733,67429302aad14e89,2020-03-20 12:26:49.970203,2020-03-20 12:26:49.970263
4,1223e9d08e7b4997a0f95daf1d9b044f,67429302aad14e89,2020-03-20 14:00:31.726087,2020-03-20 14:00:31.726191


## Function to print the value of each matrix

In [3]:
def pt_mat(Dict, store_id):
    if store_id in Dict.keys():
        print(store_id)
        print(Dict[store_id])

## Function to store the data in dictionary, Key: store_id, Value: The specific matrix

In [4]:
def df_dict(Dict, temp):
    #temp- dataframe
    Dict= {}
    for i in range(len(temp)):
        key= temp['store_id'][i]
        value= [temp['freq'][i]]
        if  temp['store_id'][i] in Dict.keys():
            Dict[temp['store_id'][i]].append(value)
    
        else:
            key= temp['store_id'][i]
            value= [temp['freq'][i]]
            Dict[key] = value
    
    return Dict

## For filtering the data according to the specified dates

In [5]:
def filt_by_date(start_date_obj, end_date_obj, df):
    mask = (df['created_on'] > start_date_obj) & (df['created_on'] <= end_date_obj)
    df = df.loc[mask]
    return df


# Main function which will calculate the value of Each Matrix 

### Matrices: 
### 1. Number of Orders with exact baskets
### 2. Percentage of total Orders with exact baskets
### 3. Number of Orders with same order amounts (within 5% variance)
### 4. Percentage of Orders with same order amounts (within 5% variance)
### 5. Number of Orders from same location (And different from store location)
### 6. Number of Orders from same device – (simply duplicates of same device orders)
### 7. Difference in Order Value over the past
### 8. Multiple customers ordering from the same device – (customer id per device)

In [6]:

def fraud_matrix_calc ( store_id,start_date_str, end_date_str):

    # Converting the string to dateTime format 
    start_date_obj = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date_obj = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')
    march_orders['created_on']= pd.to_datetime(march_orders['created_on'])

    #Making copy of March_orders
    march_orders_lock = march_orders

    #The march_orders contains now only the rows which lie between start and end date//
    #mask = (march_orders_lock['created_on'] > start_date_obj) & (march_orders_lock['created_on'] <= end_date_obj)

    #march_orders_lock = march_orders_lock.loc[mask]
    #march_orders_lock
 
    ##########################################################################
    #Calculation of Exact amount
    march_orders_lock = filt_by_date(start_date_obj, end_date_obj, march_orders)
    march_orders_lock.head()    
    march_orders_dup = march_orders_lock[march_orders_lock.duplicated(subset=["bill_total","store_id"], keep=False)]
    march_orders_dup = march_orders_dup.pivot_table(index=['bill_total','store_id'], aggfunc='size').sort_values(ascending= False)
    #Converting from series to dataframe
    march_orders_dup = march_orders_dup.to_frame()
    #NAming third column
    march_orders_dup = march_orders_dup.rename(columns = {0: "freq"})
    #Setting the Index
    march_orders_dup.reset_index(inplace= True)
    march_orders_dup.head()

    
 
    # Converting  a dataframe to dictionary manually, dict contains the all the exact amount with respective order id as the key


    ################################      
    Dict_dup_amt = {}
    march_orders_dup_temp = march_orders_dup
    Dict_dup_amt =  df_dict(Dict_dup_amt, march_orders_dup_temp)
    pt_mat(Dict_dup_amt, store_id)
    #Dict_dup_amt CONTAINS ANSWER
    import copy
    kict_dup_amt = copy.deepcopy(Dict_dup_amt)
    hict = copy.deepcopy(Dict_dup_amt)

    # CALCULATION OF SUM OF ALL FREQUECNCIES IN DICT KICT
    for key, value in kict_dup_amt.items():
       value= sum(value)
       kict_dup_amt[key]= value
       march_orders_dup_temp = march_orders_dup_temp.reset_index(drop=True)
    #################################################      
    # CALCULATING % OF SAME ORDER AMOUNT
    sict={}
    hict = copy.deepcopy(Dict_dup_amt)
    for key, value in hict.items():
       rslt_df = march_orders_dup_temp.loc[march_orders_dup_temp['store_id'] == key ]  
       toto = rslt_df['freq'].sum()
       print(toto)
       final_list = list(map(lambda x: x/toto ,value ))    
       print(final_list)
       sict[key]= final_list
    pt_mat(sict, store_id)
    #SICT CONTAINS THE % OF SAME ORDER AMOUNT    
    ################################################################################
    # Number of Orders from same location (And different from store location)
    lat_long_rows = copy.deepcopy(march_orders)
    lat_long_rows = filt_by_date(start_date_obj, end_date_obj, lat_long_rows)
    lat_long_rows.dropna(subset = ["customer_lat"], inplace=True)# to drop empty rows
    lat_long_rows = lat_long_rows[lat_long_rows.duplicated(['customer_lat'])]#we are calculating duplicate according to locations not by store id so for particulat store the order from particular location is just 1 will also be included..
    lat_long_rows = lat_long_rows[['store_id','customer_lat','customer_long']]
    lat_long_rows = lat_long_rows.reset_index(drop=True)
    lat_long_rows = lat_long_rows.pivot_table(index=["store_id","customer_lat","customer_long"], aggfunc='size').sort_values(ascending= False)
    #Converting from series to dataframe
    lat_long_rows = lat_long_rows.to_frame()
    #NAming third column
    lat_long_rows = lat_long_rows.rename(columns = {0: "freq",})
    #Setting the Index
    lat_long_rows.reset_index(inplace= True)
    lat_long_rows.head()
        
    Dict_lat_long = {}
    lat_long_rows_dup = lat_long_rows
    Dict_lat_long =  df_dict(Dict_lat_long, lat_long_rows_dup)   
    #Dict_lat_long CONTANS ANSWER
    import copy
    kict_lat_long = copy.deepcopy(Dict_lat_long)
    for key, value in kict_lat_long.items():
       value= sum(value)
       kict_lat_long[key]= value
    

    pt_mat(kict_lat_long, store_id)
    #kict_lat_long CONTAINS SUM OF VALUES OF DICT_LAT_LONG
    ############################################################################
    # CALCULATION OF MULTPLE CUSTOMERS ORDERING FROM THE SAME DEVICE
    #USING JOINS  TO MERGE DATA FRAMES
    order = pd.read_csv('H:/intern_lovelocal/Fraud Analysis/march_orders.csv')
    device = pd.read_csv('H:/intern_lovelocal/Fraud Analysis/march_customer_device_mapping.csv')
    #CONVERTING CRATED ON COLUMN TO DATATIME TYPE
    order['created_on']= pd.to_datetime(order['created_on'])
    device['created_on']= pd.to_datetime(device['created_on'])
    #NUMBER OF ROWS IN MERGE DATAFR ARE MORE THAN THE DEVICE_MAPPING DATAFRAME AND LESS THAN  MARCH_DETAILS DATA FRAME, ITS LIKE THE SOME EXTRA ROWS IN MARCH DETAILS HAD SAME CUSTOMER ID ON WHICH BASIS WE MERGED THE DATA SET
    merged_inner = pd.merge(left=device, right=order, left_on='customer_id', right_on='customer_id')
    #SELECTING THE REQUIRED COLS FORM THE MERGED DATA FRAME
    #WHEN PASSING TO MAIN FUNC IN JUPYTER NOT REM TO INCLUDE THE CREATED_ON_Y, CUSTOMER_ID, MODIFIED_ON COLUMN ALSO, FOR  NOW ITS NOT REQ AS FOR TETNG
    merged_inner = merged_inner[['store_id','order_id','device_identifier','customer_id','created_on_y']]
    mask = (merged_inner['created_on_y'] > start_date_obj) & (merged_inner['created_on_y'] <= end_date_obj)
    merged_inner = merged_inner.loc[mask]
    ####################################################################################################
    #CALCULATION OF NUMBER OF ORDERS FROM SAME DEVICE FOR EACH PARTICULAR STORE(SIMPLY DUPLICATE OF SAME DEVICE ORDERS)
    #WE WILL REMOVE THE CREATED ON COLUMN FOR NOW AS IT WILL PASSED TO MAIN FUNCTION THE DATA WILL BE AUTOMNATICALY SORTED ACCORDIG TO START AND END DATE
    #Calculation of Exact amount
    #REMOVING DUPLICATE ORDER ID ROWS BECASUE EACH ORDER HAS ITS SPECIFIC ORDER WHICH CANNOT BE ASSIGNED TO ANY OTHER ORDER
    merged_inner.drop_duplicates(subset ="order_id",keep = False, inplace = True)
    merged_inner_dup_dev = merged_inner[merged_inner.duplicated(subset=["store_id","device_identifier"], keep=False)]
    merged_inner_dup_dev = merged_inner_dup_dev.pivot_table(index=['store_id','device_identifier'], aggfunc='size').sort_values(ascending= False)
    
    #Converting from series to dataframe
    merged_inner_dup_dev = merged_inner_dup_dev.to_frame()
    #NAming third column
    merged_inner_dup_dev = merged_inner_dup_dev.rename(columns = {0: "freq"})
    #Setting the Index
    merged_inner_dup_dev.reset_index(inplace= True)
    merged_inner_dup_dev.head()
    
    #FORMING DICTIONARY
    Dict_dup_dev= {}
    temp= merged_inner_dup_dev
    for i in range(len(temp)):
        key= temp['store_id'][i]
        value= [temp['freq'][i],temp['device_identifier'][i] ]
        if  temp['store_id'][i] in Dict_dup_dev.keys():
            Dict_dup_dev[temp['store_id'][i]].append(value)
    
        else:
            key= temp['store_id'][i]
            value= [temp['freq'][i],temp['device_identifier'][i] ]
            Dict_dup_dev[key] = value
    
    pt_mat(Dict_dup_dev, store_id)
    #Dict_dup_dev contains answer 
#############################################################################################
    #MULTIPLE CUSTOMERS ORDERING FROM THE SAME DEVICE PER STORE
    # CALCULATION OF EXACT BASKET
    # FIRST STEP FOR THAT IS THE MERGING OF DATA FRAMES 
    # MERGING MARCH ORDER PRODUCT DETAUUILS WITH MARCH ORDER PRODUCTS BY MPAANI PRODUCT ID TO GET ORDER ID
    # THEN MERGING WITH MARCH ORDERS TO GET STORE ID
    # THEN NORMAL COMPUTATION
    # STEP1 
    #USING JOINS  TO MERGE DATA FRAMES
    order = pd.read_csv('H:/intern_lovelocal/Fraud Analysis/march_order_product_details.csv')
    device = pd.read_csv('H:/intern_lovelocal/Fraud Analysis/march_order_products.csv')
    #NUMBER OF ROWS IN MERGE DATAFR ARE MORE THAN THE DEVICE_MAPPING DATAFRAME AND LESS THAN  MARCH_DETAILS DATA FRAME, ITS LIKE THE SOME EXTRA ROWS IN MARCH DETAILS HAD SAME CUSTOMER ID ON WHICH BASIS WE MERGED THE DATA SET
    merged_inner_basket = pd.merge(left=device, right=order, left_on='mpaani_product_id', right_on='mpaani_product_id')
    #SELECTING THE REQUIRED COLS FORM THE MERGED DATA FRAME
    #WHEN PASSING TO MAIN FUNC IN JUPYTER NOT REM TO INCLUDE THE CREATED_ON_Y, CUSTOMER_ID, MODIFIED_ON COLUMN ALSO, FOR  NOW ITS NOT REQ AS FOR TETNG
    merged_inner_basket = merged_inner_basket[['order_id','product_name','mrp']]

    #REMOVING DUPLICATE ORDER ID ROWS BECASUE EACH ORDER HAS ITS SPECIFIC ORDER WHICH CANNOT BE ASSIGNED TO ANY OTHER ORDER
    merged_inner_basket.drop_duplicates(subset ="order_id",keep = False, inplace = True)

    # STEP 2, MERGING IT WITH MARCH ORDERS TO GET STORE ID
    mo = pd.read_csv('H:/intern_lovelocal/Fraud Analysis/march_orders.csv')
    mo['created_on']= pd.to_datetime(mo['created_on'])

    merged_inner_basket = pd.merge(left=merged_inner_basket, right=mo, left_on='order_id', right_on='order_id')
    merged_inner_basket = merged_inner_basket[['store_id','order_id','product_name','mrp','created_on']]
    merged_inner_basket = filt_by_date(start_date_obj, end_date_obj, merged_inner_basket)
    
    merged_inner_ex_basket = merged_inner_basket[merged_inner_basket.duplicated(subset=["store_id","product_name","mrp"], keep=False)]
    merged_inner_ex_basket = merged_inner_ex_basket.pivot_table(index=['store_id','product_name','mrp'], aggfunc='size').sort_values(ascending= False)
    #Converting from series to dataframe
    merged_inner_ex_basket = merged_inner_ex_basket.to_frame()
    #NAming third column
    merged_inner_ex_basket = merged_inner_ex_basket.rename(columns = {0: "freq"})
    #Setting the Index
    merged_inner_ex_basket.reset_index(inplace= True)
    merged_inner_ex_basket.head()
    #FORMING DICTIONARY
    temp = merged_inner_ex_basket 
    Dict_exact_bas= {}
    for i in range(len(temp)):
        key= temp['store_id'][i]
        value= [temp['product_name'][i] ,temp['mrp'][i],temp['freq'][i] ]
        if  temp['store_id'][i] in Dict_exact_bas.keys():
            Dict_exact_bas[temp['store_id'][i]].append(value)
    
        else:
            key= temp['store_id'][i]
            value= [temp['product_name'][i] ,temp['mrp'][i],temp['freq'][i] ]
            Dict_exact_bas[key] = value
    
    pt_mat(Dict_exact_bas, store_id)
    #THE ANSWER IS IN Dict_exact_bas
##################################################################################################


## Below the user has to input the value, Store_id: for the specific store one wants to get the matrix values
## Dates: from specific to start to end of dates between which the matrix values will be calculated

In [9]:
#Calling the function
store_id = 314491        
start_date_str ='2020-03-3'
end_date_str = '2020-03-24'

## Calling main funciton(Martrix fraud detection)

In [ ]:
fraud_matrix_calc(store_id, start_date_str, end_date_str)